# Blockhash Oracle Test Script

This notebook tests the deployed oracle system by sending messages and requesting block hashes.

## 1. Configuration

In [ ]:
%load_ext autoreload
%autoreload 2
# Network type selection
NETWORK_TYPE = "testnets"  # "testnets" or "mainnets"

# Test configuration
# Which chain to use for read tests
READ_TEST_CHAIN = "base-sepolia"

# Whether to test broadcasting
TEST_BROADCAST = True

## 2. Initialize Environment

In [ ]:
import json
import os
import sys
import logging
import subprocess
import time
from pathlib import Path

from dotenv import load_dotenv
from eth_account import Account
from web3 import Web3

# Add parent directory to path for imports
sys.path.append(str(Path().resolve().parent))
from ABIs import endpointV2_abi

# Import from deployment folder
from LZMetadata import LZMetadata
from DeploymentManager import DeploymentManager
from utils import encode_headers

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Load environment variables
load_dotenv()

# Constants
READ_CHANNEL_ID = 4294967295  # max uint32

## 3. Helper Functions

In [ ]:
def get_vyper_abi(filepath):
    """Get ABI from Vyper contract file"""
    command = ["vyper", filepath, "-f", "abi_python"]
    try:
        result = subprocess.run(command, capture_output=True, text=True, check=True)
        return result.stdout
    except subprocess.CalledProcessError as e:
        return f"Error: {e.stderr}"


def send_tx_single(w3, func, acc, value=0, gas=0):
    """Send transaction with retry logic"""

    try:
        tx = func.build_transaction(
            {
                "from": acc.address,
                "nonce": w3.eth.get_transaction_count(acc.address),
                "value": value,
            }
        )
    except Exception:
        tx = func.build_transaction(
            {
                "from": acc.address,
                "nonce": w3.eth.get_transaction_count(acc.address),
                "value": value,
                "gasPrice": int(1.1 * w3.eth.gas_price),
            }
        )
    if gas > 0:
        tx["gas"] = gas
    else:
        try:
            tx["gas"] = int(w3.eth.estimate_gas(tx) * 1.2)
        except Exception as e:
            if "no data" in str(e):
                tx["gas"] = 1_000_000
            else:
                raise (e)

    signed_tx = w3.eth.account.sign_transaction(tx, private_key=acc.key)
    tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
    receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    if receipt.status == 0:
        raise Exception("Transaction failed")
    return tx_hash


def send_tx(w3, func, acc, value=0, gas=0):
    """Send transaction with retry logic"""
    success = False
    while not success:
        try:
            tx_hash = send_tx_single(w3, func, acc, value, gas)
            success = True
        except Exception as e:
            if (
                "replacement transaction underpriced" in str(e)
                or "nonce too low" in str(e)
                or "could not replace existing tx" in str(e)
            ):
                print(str(e), "Retrying...")
                success = False
                time.sleep(1)
            else:
                raise e
    return tx_hash

## 4. Load Deployment and Configuration

In [ ]:
# Load deployment state
deployment_manager = DeploymentManager()
deployed_contracts = deployment_manager.get_all_deployed_contracts(NETWORK_TYPE)

if not deployed_contracts:
    raise ValueError(f"No deployments found for {NETWORK_TYPE}")

logging.info(f"Found deployments for {len(deployed_contracts)} chains")

# Load chains configuration
with open("../chain-parse/chains.json", "r") as f:
    chains_config = json.load(f)

all_chains = chains_config[NETWORK_TYPE]
chains = all_chains
# Find main chain
main_chain = None
for chain_name, config in all_chains.items():
    if config.get("is_main_chain", False):
        main_chain = chain_name
        break

if not main_chain:
    raise ValueError(f"No main chain defined for {NETWORK_TYPE}")

logging.info(f"Main chain: {main_chain}")

## 5. Setup Account

In [ ]:
# Get deployer account
if NETWORK_TYPE == "testnets":
    private_key = os.environ.get("WEB3_TESTNET_PK")
    if not private_key:
        raise ValueError("WEB3_TESTNET_PK not found in environment")
    account = Account.from_key(private_key)
else:
    # For mainnets, use secure key utilities
    sys.path.append(os.path.expanduser("~/projects/keys/scripts"))
    from secure_key_utils import get_web3_account
    from getpass import getpass

    ENCRYPTED_PK = os.environ.get("ENCRYPTED_PK")  # encrypted blob
    account = get_web3_account(ENCRYPTED_PK, getpass())  # decrypts with local pass + keychain

logging.info(f"Deployer address: {account.address}")

## 6. Initialize State with Deployed Contracts

In [ ]:
from web3.middleware import ExtraDataToPOAMiddleware

# Initialize LayerZero metadata
lz = LZMetadata()

# Initialize state dictionary
state_dict = {}

# Get ABIs
ABI_RELAY = get_vyper_abi("../../contracts/messengers/LZBlockRelay.vy")
ABI_ORACLE = get_vyper_abi("../../contracts/BlockOracle.vy")
ABI_HEADER_VERIFIER = get_vyper_abi("../../contracts/HeaderVerifier.vy")

# Load all deployed chains (including main chain if needed for read config)
all_deployed_chains = set(deployed_contracts.keys())
if main_chain in all_deployed_chains and main_chain not in chains:
    chains[main_chain] = all_chains[main_chain]

for chain_name, config in chains.items():
    if chain_name not in deployed_contracts:
        logging.warning(f"No contracts deployed on {chain_name}, skipping")
        continue

    state_dict[chain_name] = {}

    ankr_key = os.environ.get("ANKR_API_KEY")
    drpc_key = os.environ.get("DRPC_API_KEY")

    # Build RPC URL
    if config.get("drpc") is not None:
        rpc_url = config["drpc"].format(drpc_key)
    elif config.get("public") is not None:
        rpc_url = config["public"]
    elif config.get("ankr") is not None:
        rpc_url = config["ankr"].format(ankr_key)
    elif config.get("rpc") is not None:
        rpc_url = config["rpc"]
    else:
        raise ValueError(f"No RPC URL found for {chain_name}")

    # Store state
    state_dict[chain_name]["config"] = config
    state_dict[chain_name]["rpc"] = rpc_url
    state_dict[chain_name]["w3"] = Web3(Web3.HTTPProvider(rpc_url))
    state_dict[chain_name]["w3"].middleware_onion.inject(ExtraDataToPOAMiddleware, layer=0)

    state_dict[chain_name]["evm_version"] = config["evm_version"]

    # Get LayerZero metadata
    try:
        lz_metadata = lz.get_chain_metadata(chain_name)
        state_dict[chain_name]["eid"] = lz_metadata["metadata"]["eid"]
        state_dict[chain_name]["endpoint"] = lz_metadata["metadata"]["endpointV2"]
        state_dict[chain_name]["send_lib"] = lz_metadata["metadata"].get(
            "sendUln302", "unavailable"
        )
        state_dict[chain_name]["receive_lib"] = lz_metadata["metadata"].get(
            "receiveUln302", "unavailable"
        )
        state_dict[chain_name]["read_lib"] = lz_metadata["metadata"].get(
            "readLib1002", "unavailable"
        )
        state_dict[chain_name]["dvns"] = lz_metadata["dvns"]
        state_dict[chain_name]["executor"] = lz_metadata["metadata"].get(
            "executor", "0x0000000000000000000000000000000000000000"
        )
    except Exception as e:
        logging.warning(f"Failed to get LZ metadata for {chain_name}: {e}")

    # Load deployed contracts
    contracts = deployed_contracts[chain_name]
    # Load contracts
    # Other chains have BlockOracle, HeaderVerifier, and LZBlockRelay
    if chain_name == main_chain:
        continue
    state_dict[chain_name]["block_relay_w3"] = state_dict[chain_name]["w3"].eth.contract(
        address=contracts["LZBlockRelay"], abi=ABI_RELAY
    )
    state_dict[chain_name]["oracle_w3"] = state_dict[chain_name]["w3"].eth.contract(
        address=contracts["BlockOracle"], abi=ABI_ORACLE
    )
    state_dict[chain_name]["header_verifier_w3"] = state_dict[chain_name]["w3"].eth.contract(
        address=contracts["HeaderVerifier"], abi=ABI_HEADER_VERIFIER
    )
    state_dict[chain_name]["endpoint_w3"] = state_dict[chain_name]["w3"].eth.contract(
        address=state_dict[chain_name]["endpoint"], abi=endpointV2_abi
    )

logging.info("Loaded all deployed contracts")

## 7. Check Current Oracle State

In [ ]:
# Check oracle data on all chains
print("\n" + "=" * 80)
print("CURRENT ORACLE STATE")
print("=" * 80)

# Get main chain block number
main_block = state_dict[main_chain]["w3"].eth.block_number
print(f"\nCurrent block on {main_chain}: {main_block}")

# Check oracle state on all other chains
for chain_name in state_dict.keys():
    if chain_name == main_chain:
        continue

    oracle_w3 = state_dict[chain_name]["oracle_w3"]

    try:
        last_block = oracle_w3.functions.last_confirmed_block_number().call()
        if last_block > 0:
            block_hash = oracle_w3.functions.get_block_hash(last_block).call()
            print(f"\n{chain_name}:")
            print(f"  Last confirmed block: {last_block}")
            print(f"  Block hash: {block_hash.hex()}")
            print(f"  Blocks behind: {main_block - last_block}")
        else:
            print(f"\n{chain_name}: No blocks confirmed yet")
    except Exception as e:
        print(f"\n{chain_name}: Error reading oracle state - {e}")

## 8. Test Simple Read (Without Broadcast)

In [ ]:
# Test read on configured chain
READ_TEST_CHAIN = "base-sepolia"
if READ_TEST_CHAIN not in state_dict:
    print(f"Read test chain {READ_TEST_CHAIN} not found in deployment")
elif state_dict[READ_TEST_CHAIN]["read_lib"] == "unavailable":
    print(f"{READ_TEST_CHAIN} does not have read capability")
else:
    print(f"\nTesting read on {READ_TEST_CHAIN}...")

    relay_w3 = state_dict[READ_TEST_CHAIN]["block_relay_w3"]
    w3 = state_dict[READ_TEST_CHAIN]["w3"]

    # Quote read fee
    read_gas = 150_000
    fee = relay_w3.functions.quote_read_fee(read_gas, 0).call()
    print(f"Read fee: {fee / 1e18:.6f} ETH")

    # Request block hash (0 = latest)
    request_block = 0
    print(f"Requesting block {request_block} (latest)...")

    func = relay_w3.functions.request_block_hash([], [], 0, read_gas, request_block)
    tx_hash = send_tx(w3, func, account, int(1.2 * fee))
    print(f"Transaction sent: {tx_hash.hex()}")

## 9. Test Broadcast

In [ ]:
READ_TEST_CHAIN = "base-sepolia"
broadcaster_w3 = state_dict[READ_TEST_CHAIN]["block_relay_w3"]
# Get all configured peers
all_eids = [state_dict[key]["eid"] for key in state_dict.keys()]
receive_eids = []

for eid in all_eids:
    if eid == state_dict[READ_TEST_CHAIN]["eid"] or eid == state_dict[main_chain]["eid"]:
        continue
    # Check if peer is configured
    peer = broadcaster_w3.functions.peers(eid).call()
    if int.from_bytes(peer, "big") != 0:
        receive_eids.append(eid)

print(receive_eids)

In [ ]:
total_fees = 0
for eid in receive_eids:
    receive_gas = 150_000
    try:
        broadcast_fees = broadcaster_w3.functions.quote_broadcast_fees([eid], receive_gas).call()
        total_fees += broadcast_fees[0]
    except Exception as e:
        print(f"Error getting broadcast fees for {eid}: {e}")
        continue
    print(f"Broadcast fees for {eid}: {broadcast_fees}")
print(f"Total fees: {total_fees/1e18}")

In [ ]:
if TEST_BROADCAST and READ_TEST_CHAIN in state_dict:
    print(f"\nTesting broadcast from {READ_TEST_CHAIN}...")

    broadcaster_w3 = state_dict[READ_TEST_CHAIN]["block_relay_w3"]
    w3 = state_dict[READ_TEST_CHAIN]["w3"]

    # Get all configured peers
    all_eids = [state_dict[key]["eid"] for key in state_dict.keys()]
    receive_eids = []

    for eid in all_eids:
        if eid == state_dict[READ_TEST_CHAIN]["eid"] or eid == state_dict[main_chain]["eid"]:
            continue
        # Check if peer is configured
        peer = broadcaster_w3.functions.peers(eid).call()
        if int.from_bytes(peer, "big") != 0:
            receive_eids.append(eid)

    if receive_eids:
        print(f"Broadcasting to {len(receive_eids)} chains...")

        # Quote broadcast fees
        receive_gas = 150_000
        broadcast_fees = broadcaster_w3.functions.quote_broadcast_fees(
            receive_eids, receive_gas
        ).call()
        total_fee = sum(broadcast_fees)
        print(f"Total broadcast fee: {total_fee / 1e18:.6f} ETH")

        # Broadcast latest block
        func = broadcaster_w3.functions.broadcast_latest_block(
            receive_eids, broadcast_fees, receive_gas
        )
        tx_hash = send_tx(w3, func, account, total_fee)
        print(f"Broadcast transaction sent: {tx_hash.hex()}")
    else:
        print("No peers configured for broadcasting")
else:
    print("\nSkipping broadcast test")

## 10. Test Read with Broadcast

In [ ]:
READ_TEST_CHAIN = "base-sepolia"
if (
    TEST_BROADCAST
    and READ_TEST_CHAIN in state_dict
    and state_dict[READ_TEST_CHAIN]["read_lib"] != "unavailable"
):
    print(f"\nTesting read with broadcast from {READ_TEST_CHAIN}...")

    broadcaster_w3 = state_dict[READ_TEST_CHAIN]["block_relay_w3"]
    w3 = state_dict[READ_TEST_CHAIN]["w3"]

    # Get receive chains
    all_eids = [state_dict[key]["eid"] for key in state_dict.keys()]
    receive_eids = []

    for eid in all_eids:
        if eid == state_dict[READ_TEST_CHAIN]["eid"] or eid == state_dict[main_chain]["eid"]:
            continue
        peer = broadcaster_w3.functions.peers(eid).call()
        if int.from_bytes(peer, "big") != 0:
            receive_eids.append(eid)

    if receive_eids:
        # Quote fees
        receive_gas = 150_000
        broadcast_fees = broadcaster_w3.functions.quote_broadcast_fees(
            receive_eids, receive_gas
        ).call()
        broadcast_fees = [int(fee * 1.01) for fee in broadcast_fees]  # Add 1% buffer

        BROADCAST_GAS = 5_000_000
        read_fee_with_broadcast = broadcaster_w3.functions.quote_read_fee(
            BROADCAST_GAS, sum(broadcast_fees)
        ).call()

        print(f"Read fee with broadcast: {read_fee_with_broadcast / 1e18:.6f} ETH")
        print(f"Broadcasting to {len(receive_eids)} chains...")

        # Request block hash with broadcast
        func = broadcaster_w3.functions.request_block_hash(
            receive_eids, broadcast_fees, receive_gas, BROADCAST_GAS
        )
        tx_hash = send_tx(w3, func, account, value=read_fee_with_broadcast, gas=5_000_000)
        print(f"Read + broadcast transaction sent: {tx_hash.hex()}")
        print(
            f"\nThis will read the latest block from {main_chain} and broadcast to all configured chains"
        )
    else:
        print("No peers configured for broadcasting")
else:
    print("\nSkipping read with broadcast test")

## 11. Submit Block Header Test

In [ ]:
# Test submitting a block header
test_chain = READ_TEST_CHAIN if READ_TEST_CHAIN in state_dict else list(state_dict.keys())[1]
for test_chain in state_dict.keys():
    if test_chain != main_chain:
        time.sleep(3)
        print(f"\nTesting block header submission on {test_chain}...")

        oracle_w3 = state_dict[test_chain]["oracle_w3"]
        header_verifier_w3 = state_dict[test_chain]["header_verifier_w3"]

        # Get last confirmed block
        block_number = oracle_w3.functions.last_confirmed_block_number().call()

        if block_number > 0:
            print(f"Last confirmed block: {block_number}")

            # Get current header data
            headers = oracle_w3.functions.block_header(block_number).call()
            print("\nCurrent block header data:")
            print(f"  Block hash: {headers[0].hex()}")
            print(f"  Parent hash: {headers[1].hex()}")
            print(f"  State root: {headers[2].hex()}")
            print(f"  Receipt root: {headers[3].hex()}")
            print(f"  Block number: {headers[4]}")
            print(f"  Timestamp: {headers[5]}")

            # Get block data from main chain
            main_w3 = state_dict[main_chain]["w3"]
            block_data = main_w3.eth.get_block(block_number, full_transactions=False)

            # Encode headers
            encoded_headers = encode_headers(block_data)
            print(f"\nEncoded headers: {encoded_headers.hex()[:100]}...")

            # Submit header
            try:
                w3 = state_dict[test_chain]["w3"]
                func = header_verifier_w3.functions.submit_block_header(
                    oracle_w3.address, encoded_headers
                )
                tx_hash = send_tx(w3, func, account)
                print(f"\nHeader submission transaction: {tx_hash.hex()}")
            except Exception as e:
                print(f"\nError submitting header: {e}")
                if "already submitted" in str(e).lower():
                    print("Header was already submitted for this block")
        else:
            print("No confirmed blocks yet - run a read test first")
    else:
        print("\nSkipping header submission test on main chain")

## 12. Final State Check

In [ ]:
# Wait a bit for transactions to process
# Check final oracle state
print("\n" + "=" * 80)
print("FINAL ORACLE STATE")
print("=" * 80)

# Get main chain block number
main_block = state_dict[main_chain]["w3"].eth.block_number
print(f"\nCurrent block on {main_chain}: {main_block}")

# Check oracle state on all other chains
for chain_name in state_dict.keys():
    if chain_name == main_chain:
        continue

    oracle_w3 = state_dict[chain_name]["oracle_w3"]

    try:
        last_block = oracle_w3.functions.last_confirmed_block_number().call()
        if last_block > 0:
            block_hash = oracle_w3.functions.get_block_hash(last_block).call()
            print(f"\n{chain_name}:")
            print(f"  Last confirmed block: {last_block}")
            print(f"  Block hash: {block_hash.hex()}")
            print(f"  Blocks behind: {main_block - last_block}")

            # Check if state root is available
            try:
                state_root = oracle_w3.functions.get_state_root(last_block).call()
                if state_root != b"\x00" * 32:
                    print(f"  State root: {state_root.hex()}")
                else:
                    print("  State root: Not available")
            except Exception as e:
                print(f"  Error getting state root: {e}")
                pass
        else:
            print(f"\n{chain_name}: No blocks confirmed yet")
    except Exception as e:
        print(f"\n{chain_name}: Error reading oracle state - {e}")